# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Álvaro Velasco Romero<br>
Url: https://colab.research.google.com/drive/1pDtwtgFYTkedEHh8ie7yi8zUsanAWwGH?usp=sharing<br>
GitHub: https://github.com/AVR185/03MIAR---Algoritmos-de-Optimizacion/tree/main/SEMINARIO<br>


Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema:

> Importar todas las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import copy
import random
import itertools

##Problema 2 - Enunciado.

**Organizar los horarios de partidos de La Liga**
* Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.
* Los horarios disponibles se conocen a priori y son los siguientes:

<br>

| Día | Horario |
|:---------|:-----|
| Viernes | 20 |
| Sábado | 12, 16, 18, 20 |
| Domingo | 12, 16, 18, 20 |
| Lunes | 20 |

<br>

* En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la categoría A, 11 equipos de categoría B y 6 equipos de categoría C.
* Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

<br>

|| Categoría A | Categoría B | Categoría C |
|:---------|:-----|:-----|:-----|
| **Categoría A** |2 Millones|1,3 Millones|1 Millón|
| **Categoría B** ||0,9 Millones|0,75 Millones|
| **Categoría C** ||| 0,47 Millones |

<br>

* Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla
* Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

<br>

|| Viernes | Sábado | Domingo | Lunes |
|:---------|:-----|:-----|:-----|:-----|
| **12 h** |-|0,55|0,45|-|
| **16 h** |-|0,7|0,75|-|
| **18 h** |-|0,8|0,85|-|
| **20 h** |0,4|1|1|0,4|

<br>

* Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la
siguiente tabla dependiendo del número de coincidencias:

<br>

| Coincidencias | % |
|:---------|:-----|
| 0 |0%|
| 1 |25%|
| 2 |45%|
| 3 |60%|
| 4 |70%|
| 5 |75%|
| 6 |78%|
| 7 |80%|
| 8 |80%|

> Datos del enunciado

In [2]:
# Configuración de los datos
equipos = {"Real Madrid": "A", "Barcelona": "A", "R. Sociedad": "A",
           "Celta": "B", "Valencia": "B", "Athletic": "B", "Villarreal": "B", "Alavés": "B", "Levante": "B",
           "Espanyol": "B", "Sevilla": "B", "Betis": "B", "Atlético": "B", "Getafe": "B",
           "Mallorca": "C", "Eibar": "C", "Leganés": "C", "Osasuna": "C", "Granada": "C", "Valladolid": "C"}

ponderacion_por_coincidencia = {0: 1, 1: 1, 2: 0.75, 3: 0.55, 4: 0.4, 5: 0.3, 6: 0.25, 7: 0.22, 8: 0.2, 9: 0.2, 10: 0.2}

audiencia_enfrentamientos = pd.DataFrame([[2, 1.3, 1],[1.3, 0.9, 0.75],[1, 0.75, 0.47]],
                                         columns=["A", "B", "C"], index=["A", "B", "C"])
display(audiencia_enfrentamientos)

tabla_partidos = pd.DataFrame({"Ponderacion": [0.4,0.55,0.7,0.8,1,0.45,0.75,0.85,1,0.4], "Audiencia": [0,0,0,0,0,0,0,0,0,0]},
                              columns=["Partidos", "Categorias", "Base", "Ponderacion", "Corr. Coincidencia", "Audiencia"],
                              index=["Viernes 20h", "Sabado 12h", "Sabado 16h", "Sabado 18h", "Sabado 20h", "Domingo 12h", "Domingo 16h", "Domingo 18h", "Domingo 20h", "Lunes 20h"])

tabla_partidos = tabla_partidos.sort_values(by=['Ponderacion'], ascending=False)
display(tabla_partidos)


,A,B,C
A,2.0,1.30,1.00
B,1.3,0.90,0.75
C,1.0,0.75,0.47


,Partidos,Categorias,Base,Ponderacion,Corr. Coincidencia,Audiencia
Sabado 20h,NaN,NaN,NaN,1.00,NaN,0
Domingo 20h,NaN,NaN,NaN,1.00,NaN,0
Domingo 18h,NaN,NaN,NaN,0.85,NaN,0
Sabado 18h,NaN,NaN,NaN,0.80,NaN,0
Domingo 16h,NaN,NaN,NaN,0.75,NaN,0
Sabado 16h,NaN,NaN,NaN,0.70,NaN,0
Sabado 12h,NaN,NaN,NaN,0.55,NaN,0
Domingo 12h,NaN,NaN,NaN,0.45,NaN,0
Viernes 20h,NaN,NaN,NaN,0.40,NaN,0
Lunes 20h,NaN,NaN,NaN,0.40,NaN,0


## Pregunta 1.
> (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>


Se trata de un caso de **Variación con Repetición** ya que disponemos de un conjunto de 10 horarios que podemos asignar a 10 partidos diferentes, pudiendo asignar el mismo horario a todos los partidos y teniendo en cuenta que el orden influye.<br>
Es decir, para cada partido hay 10 posibles horarios.
La fórmula es la siguiente:<br>
<br>
$VR_{n,k} = n^{k} = 10^{10} = 10.000.000.000$ posibilidades
<br>

> ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones<br>

Las restricciones de este problema son que siempre debe haber al menos un partido asignado para el viernes y otro para el lunes.<br>

Lo más sencillo para calcular el número de posibles opciones teniendo en cuenta las restricciones es calcular las combinaciones posibles en las que no hay partido el lunes, el viernes o en ambos casos a la vez y restarselo al número de combinaciones sin restricciones obtenido en el apartado anterior ($10^{10}$).<br>
<br>
$9^{10} + 9^{10} + 8^{10} = 8.047.310.626$ posibles combinaciones en las que no hay partidos ni viernes, ni lunes, ni el viernes y lunes a la vez.<br>
<br>
Finalmente obtenemos:<br>

$Total => 10^{10} - (9^{10} + 9^{10} + 8^{10}) = 1.952.689.374$ posibles combinaciones con restricciones

Despues quedan 8 partidos para disponer en 8 horarios posibles y se permite la repetición.

##Pregunta 2.
> Modelo para el espacio de soluciones<br>
(*) ¿Cuál es la estructura de datos que mejor se adapta al problema? Argumentalo. (Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


La mejor forma de representar la solución es por medio de una tabla o matriz, pudiendo disponer de una columna para los partidos y otra para el horario asignado a cada uno de ellos. Además, se puede calcular una columna "Espectadores" de cada franja horaria a partir de la ponderación de dicho horario y el número de espectadores esperado según la caterogría de cada partido disputado.

Finalmente si sumamos todos los valores de esta última columna obtendríamos el número de espectadores total para la jornada.

Anteriormente había considerado la opción de un grafo, pero lo descarté porque no me permite visualizar y almacenar tanta información.

##Pregunta 3.
> Según el modelo para el espacio de soluciones<br>
(*) ¿Cuál es la función objetivo?<br>



Para resolver el problema debemos calcular el número de espectadores totales de una jornada de liga, que será el sumatorio de la audiencia registrada en cada franja horaria.

Por lo tanto tendremos la siguiente fórmula:

$$\sum_{i=1}^n \ b \cdot p1 \cdot p2$$

Donde:
<ul>
<li><i>b</i>: es la audiencia base de un partido en función de su categoría.</li>
<li><i>p1</i>: es la ponderación que hay que aplicar a la audiencia base en función de la franja horaria en la que se situe.</li>
<li><i>p2</i>: es la ponderación que hay que aplicar a la audiencia en función del número de partidos que coinciden en la misma franja horaria.</li>
</ul>

> (*) ¿Es un problema de maximización o minimización?<br>

Es un problema de maximización, puesto que debemos buscar la mejor distribución de partidos en las franjas horarias facilitadas que nos permitan obtener el máximo número de espectadores de la jornada.

##Pregunta 4.
> Diseña un algoritmo para resolver el problema por fuerza bruta

Hay que tener en cuenta que el sistema no es capaz de obtener todos los casos posibles ya que da un error de memoria antes de completarse la ejecución. Como se verá en un apartado posterior, estamos hablando de miles de millones de posibilidades, por ese motivo en el caso de fuerza bruta solo estudiaremos las audiencias que se obtienen repartiendo los partidos en cada franja horaria, sin posibilidad de que coincidan varios partidos a la misma hora.

Este supuesto realmente nos ayuda, ya que teniendo en cuenta la ponderación por coincidencia de partidos en un mismo horario indicada en el enunciado, se puede comprobar que no se pierde ningún caso prometedor ya que siempre será más óptimo distribuir los 10 partidos sin producir coincidencias.

No obstante, en posteriores puntos cuando se diseñe otro algoritmo que resuelva el problema, ahí si que se considerará esta posibilidad.<br>
<br>
<i>*NOTA: hay 25.200 combinaciones posibles para este estudio y se itera sobre todas ellas, por lo tanto la ejecución de la siguiente celda puede demorarse 3 minutos en completarse.</i>

In [3]:
%time
# Primero obtenemos todas las combinaciones posibles
lista_partidos = [["Celta", "Real Madrid"],
                  ["Valencia", "R. Sociedad"],
                  ["Mallorca", "Eibar"],
                  ["Athletic", "Barcelona"],
                  ["Leganés", "Osasuna"],
                  ["Villarreal", "Granada"],
                  ["Alavés", "Levante"],
                  ["Espanyol", "Sevilla"],
                  ["Betis", "Valladolid"],
                  ["Atlético", "Getafe"]]

# Función auxiliar para establecer la categoría de cada partido
def establecer_categorias(lista):
  aux = []
  partidos = copy.deepcopy(lista)   # Copiamos la lista de lista para no modificarla
  for partido in partidos:          # Obtenemos los datos de Categoria y Audiencia de cada partido
    categoria1 = equipos[partido[0]]
    categoria2 = equipos[partido[1]]
    aux.append(f"{categoria1}-{categoria2}")
    partido.append(f"{categoria1}-{categoria2}")
  return aux, partidos

categorias, lista_partidos = establecer_categorias(lista_partidos)
print(f"Categorías de los partidos de la jornada: {categorias}")

permutacion_con_repeticion = [v for v in itertools.permutations(categorias, 10)]
permutacion = set(permutacion_con_repeticion) # quitamos los casos repetidos
print(f"Total de posibilidades de ordenar los partidos, sin repetir horarios: {len(permutacion)}") # El total de casos que vamos a comprobar

# Función principal
def fuerza_bruta(lista):
  tabla = tabla_partidos.copy()
  best = tabla_partidos.copy()
  for categoria in lista:
    for i in range(len(categoria)):
      tabla.iloc[i, 1] = categoria[i] # categoria
      tabla.iloc[i, 2] = audiencia_enfrentamientos.loc[categoria[i].split("-")[0], categoria[i].split("-")[1]] # base
      tabla.iloc[i, 4] = 1 # ponderacion por coincidencia
      tabla.iloc[i, 5] = tabla.iloc[i, 2] * tabla.iloc[i, 3] * tabla.iloc[i, 4] # Audiencia

    if sum(tabla["Audiencia"]) > sum(best["Audiencia"]): best = tabla.copy()

  return best

tabla_resultado = fuerza_bruta(permutacion)
print(f"\nEspectadores totales de la jornada: {round(sum(tabla_resultado['Audiencia']) * 1000000)}\n")
display(tabla_resultado)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
Categorías de los partidos de la jornada: ['B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B']
Total de posibilidades de ordenar los partidos, sin repetir horarios: 25200

Espectadores totales de la jornada: 6856000



,Partidos,Categorias,Base,Ponderacion,Corr. Coincidencia,Audiencia
Sabado 20h,NaN,B-A,1.3,1.00,1,1.3000
Domingo 20h,NaN,B-A,1.3,1.00,1,1.3000
Domingo 18h,NaN,B-A,1.3,0.85,1,1.1050
Sabado 18h,NaN,B-B,0.9,0.80,1,0.7200
Domingo 16h,NaN,B-B,0.9,0.75,1,0.6750
Sabado 16h,NaN,B-B,0.9,0.70,1,0.6300
Sabado 12h,NaN,B-C,0.75,0.55,1,0.4125
Domingo 12h,NaN,B-C,0.75,0.45,1,0.3375
Viernes 20h,NaN,C-C,0.47,0.40,1,0.1880
Lunes 20h,NaN,C-C,0.47,0.40,1,0.1880


Hemos obtenido una tabla que en función de la categoría de los partidos nos indica la audiencia máxima de la jornada, pero faltaría concretar los partidos que se verán a cada hora:

In [4]:
lista = copy.deepcopy(lista_partidos)
for i, row in tabla_resultado.iterrows():
  for x in lista:
    if row["Categorias"] == x[2]:
      tabla_resultado.loc[i, "Partidos"] = f"{x[0]} - {x[1]}"
      x[2] = "" # Para no repetir partidos ya asignados
      break

display(tabla_resultado)

,Partidos,Categorias,Base,Ponderacion,Corr. Coincidencia,Audiencia
Sabado 20h,Celta - Real Madrid,B-A,1.3,1.00,1,1.3000
Domingo 20h,Valencia - R. Sociedad,B-A,1.3,1.00,1,1.3000
Domingo 18h,Athletic - Barcelona,B-A,1.3,0.85,1,1.1050
Sabado 18h,Alavés - Levante,B-B,0.9,0.80,1,0.7200
Domingo 16h,Espanyol - Sevilla,B-B,0.9,0.75,1,0.6750
Sabado 16h,Atlético - Getafe,B-B,0.9,0.70,1,0.6300
Sabado 12h,Villarreal - Granada,B-C,0.75,0.55,1,0.4125
Domingo 12h,Betis - Valladolid,B-C,0.75,0.45,1,0.3375
Viernes 20h,Mallorca - Eibar,C-C,0.47,0.40,1,0.1880
Lunes 20h,Leganés - Osasuna,C-C,0.47,0.40,1,0.1880


##Pregunta 5.
> Calcula la complejidad del algoritmo por fuerza bruta

En primer lugar se realiza un bucle sobre los partidos para obtener la categoría de los mismos, y con esta misma lista y la librería "itertools" obtenemos el espacio de muestras que necesitamos. Pero se tratan de bucles simples sobre una lista de 10 elementos, con una complejidad: n.

Por lo tanto el mayor número de operaciones se realiza en la función principal donde encontramos un bucle anidado en el que se realizan 4 operaciones elementales. El problema en este caso es que no iteramos sobre una lista de 10 elementos, ahora recorremos todo el espacio de muestras en el bucle más externo lo que supone repetir 25200 veces el bucle interno sobre la lista de 10 partidos. Por eso se demora tanto la ejecución.

Finalmente tenemos un orden de complejidad: $$O (tc \cdot np)$$
Donde:
<ul>
<li><i>tc</i>, el total de casos posibles (25.200).</li>
<li><i>np</i>, número de partidos de la jornada (10).</li>
</ul>

##Pregunta 6.
> (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

He diseñado un algoritmo voraz que mejora el algoritmo de fuerza bruta ya que no exploramos todos los posibles casos ni mucho menos, de hecho solo hacemos un bucle sobre los partidos de la jornada (en total 10) y tratamos de obtener el mejor resultado en cada iteración para finalmente obtener la audiencia total de la jornada.

Es este caso de una forma casi inmediata obtenemos el mismo resultado que con el método por fuerza bruta que tarda más de 3 minutos en completarse, con lo que queda demostrada la eficacia del algoritmo voraz para este problema.

In [5]:
# Función para obtener el pivote
def partition(array, aux, low, high):
    pivot = aux[high] # elegir el elemento situado más a la derecha como pivote
    i = low - 1       # puntero para el elemento más pequeño
 
    # recorrer todos los elementos comparar cada elemento con el pivote
    for j in range(low, high):
        if aux[j] >= pivot:
 
            # Si se encuentra un elemento mayor que el pivote, se intercambia con el elemento menor señalado por i
            i = i + 1

            # Intercambiamos el elemento i con el j
            (aux[i], aux[j]) = (aux[j], aux[i])
            (array[i], array[j]) = (array[j], array[i])
 
    # Intercambia el elemento pivote con el elemento mayor especificado por i
    (aux[i + 1], aux[high]) = (aux[high], aux[i + 1])
    (array[i + 1], array[high]) = (array[high], array[i + 1])
 
    return i + 1 # Devuelve la posición desde la que se realiza la partición
 
# Función para implementar la ordenación por QuickSort
def quickSort(array, aux, low, high):
    if low < high:
        # Obtenemos el pivote
        pi = partition(array, aux, low, high)

        # Recursividad a derecha e izquierda del pivote
        quickSort(array, aux, low, pi - 1)
        quickSort(array, aux, pi + 1, high)


# FUNCIÓN PRINCIPAL PARA RESOLVER EL PROBLEMA
def establecer_horario(lista):
  aux = []
  partidos = copy.deepcopy(lista)   # Copiamos la lista de lista para no modificarla
  for partido in partidos:          # Obtenemos los datos de Categoria y Audiencia de cada partido
    categoria1 = equipos[partido[0]]
    categoria2 = equipos[partido[1]]
    base = audiencia_enfrentamientos.loc[categoria1, categoria2]

    partido.append(f"{categoria1}-{categoria2}")
    aux.append(base)

  quickSort(partidos, aux, 0, len(aux)-1) # Queremos ordenar los partidos en función de la audiencia que tienen

  last = 0
  for i in range(len(partidos)):
    # El mejor partido en cuanto audiencia se pone en Sabado o Domingo a las 20h y los dos peores partidos en lunes y viernes
    # Columnas = Partidos,	Categorias,	Base,	Ponderacion,	Corr. Coincidencia,	Audiencia
    if i == 0 or i == len(partidos)-1 or i == len(partidos)-2:
      tabla_partidos.iloc[i, 0] = f"{partidos[i][0]} - {partidos[i][1]}"
      tabla_partidos.iloc[i, 1] = partidos[i][2]
      tabla_partidos.iloc[i, 2] = aux[i]
      tabla_partidos.iloc[i, 4] = ponderacion_por_coincidencia[len(tabla_partidos.iloc[i, 0].split("/"))]
      tabla_partidos.iloc[i, 5] = tabla_partidos.iloc[i, 2] * tabla_partidos.iloc[i, 3] * tabla_partidos.iloc[i, 4]
    else:
      # Ultimo horario cogido: last
      a = aux[i] * tabla_partidos.iloc[last +1, 3] # Audiencia normal
      b = aux[i] * tabla_partidos.iloc[last, 3] * ponderacion_por_coincidencia[len(tabla_partidos.iloc[last, 0].split("/")) + 1] # Audiencia si fuese en la franja anterior
      c = tabla_partidos.iloc[last, 2] * tabla_partidos.iloc[last, 3] * ponderacion_por_coincidencia[len(tabla_partidos.iloc[last, 0].split("/")) + 1] # Audiencia franja pasada corregida por coincidencia

      if (a + tabla_partidos.iloc[last, 5]) < (b + c): # Si compensa poner el partido anterior a la vez que el actual
        tabla_partidos.iloc[last, 0] = f"{tabla_partidos.iloc[last, 0]} / {partidos[i][0]} - {partidos[i][1]}" # Concatenar partidos
        tabla_partidos.iloc[last, 1] = f"{tabla_partidos.iloc[last, 1]} / {partidos[i][2]}" # Concatenar categorias
        tabla_partidos.iloc[last, 2] = tabla_partidos.iloc[last, 2] + aux[i]
        tabla_partidos.iloc[last, 4] = ponderacion_por_coincidencia[len(tabla_partidos.iloc[last, 0].split("/"))]
        tabla_partidos.iloc[last, 5] = tabla_partidos.iloc[last, 2] * tabla_partidos.iloc[last, 3] * tabla_partidos.iloc[last, 4]        
      else: # Si no compensa
        last += 1
        tabla_partidos.iloc[last, 0] = f"{partidos[i][0]} - {partidos[i][1]}" # No habrá partidos anteriores en este horario
        tabla_partidos.iloc[last, 1] = partidos[i][2]
        tabla_partidos.iloc[last, 2] = aux[i]
        tabla_partidos.iloc[last, 4] = ponderacion_por_coincidencia[len(tabla_partidos.iloc[last, 0].split("/"))]
        tabla_partidos.iloc[last, 5] = tabla_partidos.iloc[last, 2] * tabla_partidos.iloc[last, 3] * tabla_partidos.iloc[last, 4]

  return sum(tabla_partidos["Audiencia"])

> Ejecutamos la función anterior y mostramos los resultados:

In [6]:
audiencia_jornada = establecer_horario(lista_partidos)

print(f"Espectadores totales de la jornada: {round(audiencia_jornada * 1000000)}\n")

display(tabla_partidos)

Espectadores totales de la jornada: 6856000



,Partidos,Categorias,Base,Ponderacion,Corr. Coincidencia,Audiencia
Sabado 20h,Celta - Real Madrid,B-A,1.3,1.00,1,1.3000
Domingo 20h,Valencia - R. Sociedad,B-A,1.3,1.00,1,1.3000
Domingo 18h,Athletic - Barcelona,B-A,1.3,0.85,1,1.1050
Sabado 18h,Alavés - Levante,B-B,0.9,0.80,1,0.7200
Domingo 16h,Espanyol - Sevilla,B-B,0.9,0.75,1,0.6750
Sabado 16h,Atlético - Getafe,B-B,0.9,0.70,1,0.6300
Sabado 12h,Betis - Valladolid,B-C,0.75,0.55,1,0.4125
Domingo 12h,Villarreal - Granada,B-C,0.75,0.45,1,0.3375
Viernes 20h,Mallorca - Eibar,C-C,0.47,0.40,1,0.1880
Lunes 20h,Leganés - Osasuna,C-C,0.47,0.40,1,0.1880


##Pregunta 7.
> (*)Calcula la complejidad del algoritmo 

Hay que tener en cuenta que se hacen dos ordenaciones antes de iterar sobre los partidos, la primera ordenación es para ordenar los horarios de mayor a menor número de espectadores y el segundo los partidos de la jornada para colocarlos de mayor a menor categoria.<br>
<br>
Teniendo en cuenta que el algoritmo de ordenación utilizado es el **quick sort** y que supondremos que la libreria Pandas ordena la columna de la tabla con un orden de complejidad similar, tenemos que una complejidad **$n \cdot log (n)$** en el mejor de los casos y de **$n^{2}$** en el peor.<br>
<br>
Por otra parte, la función principal que hemos diseñado para resolver el problema tiene un orden de complejidad **2n** ya que realizamos un primer bucle para obtener la categoria y espectadores de cada partido y otro bucle para asignar un horario a cada partido (no anidados). Ambos bucles se realizan sobre la lista de partidos de la jornada correspondiente, por lo que en nuestro caso se realizarían únicamente 10 iteraciones en los dos bucles For.<br>
<br>
Finalmente si sumamos las operaciones tenemos:<br>

**$2n \cdot log (n) + 2n$**

Con un orden de complejidad:

**$O(n \cdot log (n))$**

Donde <i>n</i> es el número de partidos de la jornada (10), mostrando una gran mejora con respecto a la complejidad del algoritmo por fuerza bruta.

##Pregunta 8.
> Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Disponemos de un diccionario cuyas claves son los equipos que compiten en la liga, por lo que ordenandolos dos a dos de forma aleatorio ya habremos diseñado una nueva jornada de liga, es decir, un nuevo juego de datos de entrada para nuestra función.

In [10]:
nombres_equipos = list(equipos.keys())
print(nombres_equipos)

equipos_random = random.sample(nombres_equipos, 20)
print(equipos_random)

def build_jornada_liga(lst):
  aux, count = [], 0
  for i in range(int(len(lst)/2)):
    aux.append([lst[count], lst[count + 1]])
    count += 2
  return aux

nueva_jornada = build_jornada_liga(equipos_random)
print(nueva_jornada)

['Real Madrid', 'Barcelona', 'R. Sociedad', 'Celta', 'Valencia', 'Athletic', 'Villarreal', 'Alavés', 'Levante', 'Espanyol', 'Sevilla', 'Betis', 'Atlético', 'Getafe', 'Mallorca', 'Eibar', 'Leganés', 'Osasuna', 'Granada', 'Valladolid']
['Barcelona', 'Athletic', 'Granada', 'Celta', 'Leganés', 'R. Sociedad', 'Sevilla', 'Atlético', 'Getafe', 'Eibar', 'Mallorca', 'Alavés', 'Valladolid', 'Osasuna', 'Real Madrid', 'Valencia', 'Betis', 'Levante', 'Villarreal', 'Espanyol']
[['Barcelona', 'Athletic'], ['Granada', 'Celta'], ['Leganés', 'R. Sociedad'], ['Sevilla', 'Atlético'], ['Getafe', 'Eibar'], ['Mallorca', 'Alavés'], ['Valladolid', 'Osasuna'], ['Real Madrid', 'Valencia'], ['Betis', 'Levante'], ['Villarreal', 'Espanyol']]


##Pregunta 9.
> Aplica el algoritmo al juego de datos generado.

Llamamos a la función principal diseñada en la pregunta 6 con el juego de datos obtenido en el apartado anterior y mostramos los resultados:

In [11]:
audiencia_jornada = establecer_horario(nueva_jornada)

print(f"Espectadores totales de la jornada: {round(audiencia_jornada * 1000000)}\n")

display(tabla_partidos)

Espectadores totales de la jornada: 6713000



,Partidos,Categorias,Base,Ponderacion,Corr. Coincidencia,Audiencia
Sabado 20h,Barcelona - Athletic,A-B,1.3,1.00,1,1.3000
Domingo 20h,Real Madrid - Valencia,A-B,1.3,1.00,1,1.3000
Domingo 18h,Leganés - R. Sociedad,C-A,1.0,0.85,1,0.8500
Sabado 18h,Sevilla - Atlético,B-B,0.9,0.80,1,0.7200
Domingo 16h,Betis - Levante,B-B,0.9,0.75,1,0.6750
Sabado 16h,Villarreal - Espanyol,B-B,0.9,0.70,1,0.6300
Sabado 12h,Granada - Celta,C-B,0.75,0.55,1,0.4125
Domingo 12h,Getafe - Eibar,B-C,0.75,0.45,1,0.3375
Viernes 20h,Mallorca - Alavés,C-B,0.75,0.40,1,0.3000
Lunes 20h,Valladolid - Osasuna,C-C,0.47,0.40,1,0.1880


##Pregunta 10.
> Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

He usado un par de referencias como apoyo, principalmente para la ordenación quick sort puesto que el algoritmo que dimos en clase no me venía bien al haber muchos elementos iguales en las listas a ordenar.

Pero en general mi guía han sido los apuntes, los ejemplos realizados en clase y las dudas resultas por el profesor de la asignatura (muchas gracias).

Quick Sort: https://www.geeksforgeeks.org/python-program-for-quicksort/<br>
Notación Big O: https://youtu.be/__vX2sjlpXU

## Pregunta 11.
> Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

En futuros pasos sería interesante cambiar los datos del enunciado y rebajar la ponderación por coincidencia en los partidos, de esta forma podríamos ver soluciones donde varios partidos compartiesen horario.

También sería interesante añadir los partidos de segunda división y que tuviesen una ponderación extra por coincidir con partidos de primera. Esto produciría que tuviésemos que tratar con el doble de partidos y se podrían añadir nuevas categorías como la D y la E.

Con estas variaciones, si incluso añadimos partidos de la liga femenina podríamos originar que aumentara tanto la complejidad del algoritmo voraz que en determinados casos compensase tratar el problema con un algoritmo heurístico como la búsqueda voraz aleatoria, obteniendo una buena solución aunque no fuese la óptima. 